In [21]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

kospi = pd.read_html('../database/stock_code/kospi.xls')[0]; df = kospi.copy()
#kosdak = pd.read_html('../database/stock_code/kosdak.xls')[0]
#df = pd.concat([df_1, df_2], axis=0, ignore_index=True)

In [22]:
df = df.rename(columns = dict(회사명='name', 종목코드='code', 상장일='origin_date'))
df = df[['name','code','origin_date']]

In [23]:
def explore_crawling(code):
    url = 'https://finance.naver.com/item/sise_day.nhn?code=' + str(code).zfill(6) + '&page=1'
    temp = pd.read_html(url)[0].dropna(axis=0)
    if len(temp) == 0:
        output = np.NaN
    else:
        output = int(temp['날짜'].iloc[0][:4])        
    return output

In [24]:
df['recent_year'] = df['code'].apply(lambda x: explore_crawling(x))

In [25]:
df.to_csv('../database/stock_code/sc_check.csv', index=False)
df.to_csv(f'../database/spare/sc_check_{today}.csv', index=False)